## Feature selection by using anova test

In [73]:
# Read the file
import pandas as pd
df1_2 = pd.read_csv("../data/raw/price_normalised3.csv")

In [74]:
df1_2

,Unnamed: 0,name,postcode,property_type,bedrooms,bathrooms,parking,schools,cost_text,train_station(m),CBD(m),population2022,population2023,population2024,population2025,income2022,income2023,income2024,income2025,price_label
0,5321,Room 6/8 Melwood Court Meadow Heights VIC 3048,3048,Apartment / Unit / Flat,1,1,0,0.3,130.000000,2131.1,19150.7,17364.0,16549.0,15773.0,15034.0,86498,88316,90176,92078,0
1,5569,30 Daly Boulevard Highton VIC 3216,3216,House,9,2,2,2.0,130.000000,2904.1,85109.1,59229.0,60753.0,62377.0,64109.0,184195,192855,201927,211432,0
2,640,62 Reserve Rd Grovedale VIC 3216,3216,House,1,2,2,1.0,140.000000,1809.3,87665.1,59229.0,60753.0,62377.0,64109.0,184195,192855,201927,211432,0
3,5570,1/57 Wanda Road Caulfield North VIC 3161,3161,House,1,1,0,0.6,140.000000,1432.7,9784.5,10418.0,10185.0,9957.0,9734.0,56454,58176,59951,61780,0
4,5571,Room 2/31 Stanley Street Frankston VIC 3199,3199,House,1,1,1,1.1,145.000000,916.1,53197.1,60500.0,59711.0,58932.0,58164.0,176387,182176,188161,194348,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6716,6157,3502/14-16 The Esplanade St Kilda VIC 3182,3182,Apartment / Unit / Flat,3,3,3,0.8,1153.846154,1923.6,7936.4,27813.0,26448.0,25151.0,23917.0,61492,63821,66238,68747,3
6717,6156,9 Lansdowne Street Blairgowrie VIC 3942,3942,House,3,2,3,3.3,1153.846154,43496.1,106670.1,2104.0,2128.0,2152.0,2177.0,44616,45495,46391,47305,3
6718,6155,901 & 902/85 Market Street South Melbourne VIC...,3205,Apartment / Unit / Flat,3,3,1,0.3,1384.615385,4576.3,30462.2,3182.0,3085.0,2990.0,2899.0,70585,72357,74173,76035,3
6719,6154,Upstairs 2C Staley Street Brunswick VIC 3056,3056,House,0,0,0,0.8,673.076923,590.8,5545.2,17310.0,16654.0,16022.0,15415.0,68842,72533,76422,80520,1


## Remove assumed outliers which may affect the performance of the model

### Assumer postcode areas which have less than 100 population and have less than average $1000 yearly income per person as outliers

In [75]:
outliers = []
for i in range(0, len(df1_2)):
    if i not in outliers:
        if df1_2['population2022'][i] < 100 or df1_2['income2022'][i] < 1000:
            outliers.append(i)

In [76]:
# number of such outliers
len(outliers)

39

In [77]:
# remove outliers
if len(outliers) > 0:
    df1_3 = df1_2.drop(df1_2.index[outliers])

In [78]:
df1_3 = df1_3.reset_index()

### Assume properties with 0 bedrooms as outliers

In [79]:
outliers2 = []
for i in range(0, len(df1_3)):
    if i not in outliers2:
        if df1_3['bedrooms'][i] == 0:
            outliers2.append(i)

In [80]:
# number of such outliers
len(outliers2)

76

In [81]:
# remove outliers
if len(outliers2) > 0:
    df1_4 = df1_3.drop(df1_3.index[outliers2])

In [82]:
df1_4.rename(columns = {'train_station(m)':'train_station', 'CBD(m)':'CBD'}, inplace = True)
df1_4

,index,Unnamed: 0,name,postcode,property_type,bedrooms,bathrooms,parking,schools,cost_text,...,CBD,population2022,population2023,population2024,population2025,income2022,income2023,income2024,income2025,price_label
0,0,5321,Room 6/8 Melwood Court Meadow Heights VIC 3048,3048,Apartment / Unit / Flat,1,1,0,0.3,130.000000,...,19150.7,17364.0,16549.0,15773.0,15034.0,86498,88316,90176,92078,0
1,1,5569,30 Daly Boulevard Highton VIC 3216,3216,House,9,2,2,2.0,130.000000,...,85109.1,59229.0,60753.0,62377.0,64109.0,184195,192855,201927,211432,0
2,2,640,62 Reserve Rd Grovedale VIC 3216,3216,House,1,2,2,1.0,140.000000,...,87665.1,59229.0,60753.0,62377.0,64109.0,184195,192855,201927,211432,0
3,3,5570,1/57 Wanda Road Caulfield North VIC 3161,3161,House,1,1,0,0.6,140.000000,...,9784.5,10418.0,10185.0,9957.0,9734.0,56454,58176,59951,61780,0
4,4,5571,Room 2/31 Stanley Street Frankston VIC 3199,3199,House,1,1,1,1.1,145.000000,...,53197.1,60500.0,59711.0,58932.0,58164.0,176387,182176,188161,194348,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6676,6715,993,4203/35 Spring St Melbourne VIC 3004,3004,Apartment / Unit / Flat,3,2,4,1.4,1038.461538,...,1621.2,8538.0,8166.0,7812.0,7473.0,82000,78618,76292,74826,3
6677,6716,6157,3502/14-16 The Esplanade St Kilda VIC 3182,3182,Apartment / Unit / Flat,3,3,3,0.8,1153.846154,...,7936.4,27813.0,26448.0,25151.0,23917.0,61492,63821,66238,68747,3
6678,6717,6156,9 Lansdowne Street Blairgowrie VIC 3942,3942,House,3,2,3,3.3,1153.846154,...,106670.1,2104.0,2128.0,2152.0,2177.0,44616,45495,46391,47305,3
6679,6718,6155,901 & 902/85 Market Street South Melbourne VIC...,3205,Apartment / Unit / Flat,3,3,1,0.3,1384.615385,...,30462.2,3182.0,3085.0,2990.0,2899.0,70585,72357,74173,76035,3


In [83]:
# df_type = df_type.pivot_table(columns='property_type',aggfunc='mean', index='postcode')
# df_type.fillna(0)

## Using ANOVA helps to determine if a categorical variable has influence on a numerical variable

#### link: https://towardsdatascience.com/statistics-in-python-using-anova-for-feature-selection-b4dc876ef4f0#:~:text=ANOVA%20is%20used%20for%20testing%20two%20variables%2C%20where%3A,categorical%20variable%202%20another%20is%20a%20numerical%20variable

### Using the statsmodels module to calculate f-score

#### if p < 0.05, this means that the categorical variable has significant influence on the numerical variable
#### if p > 0.05, this means that the categorical variable has no significant influence on the numerical variable

In [84]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Multi-way ANOVA

In [95]:
model = ols('cost_text ~ C(property_type) + C(bedrooms) + C(bathrooms) + C(parking) + schools + train_station + CBD + population2022 + income2022',
            data=df1_4).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(property_type),3.879227e+06,13.0,6.926213,1.739330e-13
C(bedrooms),2.506500e+07,10.0,58.178391,1.976891e-113
C(bathrooms),3.135935e+07,9.0,80.875806,7.243998e-143
C(parking),2.899474e+06,13.0,5.176902,2.834065e-09
schools,1.083242e+06,1.0,25.143131,5.462720e-07
train_station,9.260902e+06,1.0,214.954885,6.498031e-48
CBD,2.219482e+07,1.0,515.164228,7.460209e-110
population2022,4.965331e+06,1.0,115.250354,1.154330e-26
income2022,6.512319e+05,1.0,15.115751,1.021146e-04
Residual,2.824522e+08,6556.0,NaN,NaN


### Useful internal features: property_type, bedrooms, bathrooms, parking
### Useful external features: schools, distance to train_station, distance to CBD, population, income

## Assumption: since bedrooms, bathrooms, parking are ordinal variables, so we asume they are categorical variables

## Limtation: Anova determines with numerical variables(distance to school&train station&CBD) may not very accurately 